In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import imodelsx.process_results
from collections import defaultdict
import numpy as np
import viz
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from os.path import join
from tqdm import tqdm
import joblib
from scipy.sparse import issparse
import sys
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/tree-prompt/dummy_runs/'

r = imodelsx.process_results.get_results_df(results_dir, results_fname='params.json')
experiment_filename = '../experiments/01_fit.py'
r = imodelsx.process_results.fill_missing_args_with_default(r, experiment_filename)
# imodelsx.process_results.delete_runs_in_dataframe(
    # r[(~r.dataset_name.str.startswith('knnp')) & (r.prompt_source == 'data_demonstrations')], actually_delete=True)

100%|██████████| 105/105 [00:08<00:00, 12.61it/s]


In [2]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(r.groupby(['dataset_name', 'prompt_source']).size())
    # display(r.groupby(['dataset_name', 'prompt_source'])
    # ['checkpoint'].unique())

dataset_name          prompt_source      
emotion               manual                 7
financial_phrasebank  manual                 7
imdb                  manual                 7
knnp__agnews          data_demonstrations    7
knnp__cb              data_demonstrations    7
knnp__cr              data_demonstrations    7
knnp__dbpedia         data_demonstrations    7
knnp__mpqa            data_demonstrations    7
knnp__mr              data_demonstrations    7
knnp__rte             data_demonstrations    7
knnp__sst2            data_demonstrations    7
knnp__subj            data_demonstrations    7
knnp__trec            data_demonstrations    7
rotten_tomatoes       manual                 7
sst2                  manual                 7
dtype: int64

In [3]:
DATA_PROCESSED_DIR = os.path.expanduser('~/cost-optimal-tree/data_processed/')
r = r.sort_values(by=['dataset_name']).reset_index().drop(columns=['index'])
for k in ['n_train', 'n_test', 'n_prompts', 'n_outputs']:
    r[k] = np.nan
for i in tqdm(range(r.shape[0])):
    args = r.iloc[i]
    print(args.dataset_name, args.checkpoint)
    run = joblib.load(join(args.save_dir_unique, 'cached_dset.pkl'))

    out_dir = join(DATA_PROCESSED_DIR, args.dataset_name)
    os.makedirs(out_dir, exist_ok=True)

    X_train = run['X_train']
    X_test = run['X_test']
    y_train = run['y_train']
    y_test = run['y_test']
    feature_names = run['feature_names']

    # run data checks
    # check that X only contains ones and zeros
    if issparse(X_train):
        X_train = X_train.toarray()
        X_test = X_test.toarray()
    assert np.all(np.isin(X_train, [0, 1]))
    assert np.all(np.isin(X_test, [0, 1]))

    # remove any cols that are all the same value
    idxs_constant = np.where(np.all(X_train == X_train[0, :], axis=0))[0]
    # print(f'removing {idxs_constant.size} constant cols')
    # print('unique', np.unique(X_train))
    X_train = np.delete(X_train, idxs_constant, axis=1)
    X_test = np.delete(X_test, idxs_constant, axis=1)
    feature_names = np.delete(feature_names, idxs_constant)

    # fit simple sklearn classifier
    clf = DecisionTreeClassifier(random_state=42, max_depth=3)
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    acc_baseline = np.unique(y_test, return_counts=True)[1][0] / y_test.size
    acc_improvement = acc - acc_baseline
    print('acc improvement', acc_improvement.round(3))
    # assert acc > acc_baseline

    # export data
    X_train = X_train.astype(bool)
    X_test = X_test.astype(bool)

    joblib.dump(
        {
            'X_train': X_train,
            'X_test': X_test,
            # 'y_train': y_train,
            # 'y_test': y_test,
            'feature_names': feature_names,
            'verbalizer': args['verbalizer'],
            'acc_improvement': acc_improvement,
            'acc': acc,
        },
        join(out_dir, imodelsx.viz.CHECKPOINTS_RENAME_DICT.get(
            args.checkpoint) + '_features.pkl')
    )
    joblib.dump({
        'y_train': y_train,
        'y_test': y_test,
    }, join(out_dir, 'labels.pkl'))

    r.loc[i, 'n_train'] = X_train.shape[0]
    r.loc[i, 'n_test'] = X_test.shape[0]
    r.loc[i, 'n_prompts'] = X_train.shape[1]
    r.loc[i, 'n_outputs'] = np.unique(y_train).size

  0%|          | 0/105 [00:00<?, ?it/s]

emotion llama_7b


  1%|          | 1/105 [00:00<01:19,  1.31it/s]

acc improvement 0.354
emotion gpt2


  2%|▏         | 2/105 [00:01<01:45,  1.02s/it]

...

acc improvement 0.303
sst2 gpt2-medium


 99%|█████████▉| 104/105 [01:19<00:02,  2.49s/it]

acc improvement 0.267
sst2 gpt2


100%|██████████| 105/105 [01:21<00:00,  1.30it/s]

acc improvement 0.227


In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    metadata = r.groupby(['dataset_name', 'prompt_source', 'n_train',
                          'n_test', 'n_outputs', 'checkpoint'])[['n_prompts']].sum()
    display(metadata)
metadata.to_pickle(join(DATA_PROCESSED_DIR, 'metadata.pkl'))

n_prompts
dataset_name         prompt_source       n_train n_test  n_outputs checkpoint                         
emotion              manual              10028.0 1254.0  2.0       EleutherAI/gpt-j-6B            32.0
                                                                   gpt2                           32.0
                                                                   gpt2-large                     32.0
                                                                   gpt2-medium                    32.0
                                                                   gpt2-xl                        32.0
                                                                   llama_7b                       32.0
                                                                   meta-llama/Llama-2-7b-hf       32.0
financial_phrasebank manual              871.0   436.0   2.0       EleutherAI/gpt-j-6B            40.0
                                                                   gpt2                           35.0
                                                                   gpt2-large                     38.0
                                                                   gpt2-medium                    40.0
                                                                   gpt2-xl                        40.0
                                                                   llama_7b                       40.0
                                                                   meta-llama/Llama-2-7b-hf       40.0
imdb                 manual              25000.0 25000.0 2.0       EleutherAI/gpt-j-6B            94.0
                                                                   gpt2                           94.0
                                                                   gpt2-large                     94.0
                                                                   gpt2-medium                    90.0
                                                                   gpt2-xl                        94.0
                                                                   llama_7b                       94.0
                                                                   meta-llama/Llama-2-7b-hf       94.0
knnp__agnews         data_demonstrations 100.0   256.0   4.0       EleutherAI/gpt-j-6B           160.0
                                                                   gpt2                          150.0
                                                                   gpt2-large                    151.0
                                                                   gpt2-medium                   143.0
                                                                   gpt2-xl                       154.0
                                                                   llama_7b                      160.0
                                                                   meta-llama/Llama-2-7b-hf      160.0
knnp__cb             data_demonstrations 100.0   56.0    3.0       EleutherAI/gpt-j-6B           120.0
                                                                   gpt2                          109.0
                                                                   gpt2-large                    119.0
                                                                   gpt2-medium                   118.0
                                                                   gpt2-xl                       117.0
                                                                   llama_7b                      120.0
                                                                   meta-llama/Llama-2-7b-hf      116.0
knnp__cr             data_demonstrations 100.0   256.0   2.0       EleutherAI/gpt-j-6B            40.0
                                                                   gpt2                           38.0
                                                                   gpt2-larg